In [7]:
import h5py
import sys
import numpy as np
import matplotlib.pyplot as plt
import os

In [9]:
# run this for 1k test---------------------------

m_mu = 105.7 #MeV
energies=[]
deposited_Es=[]
ranges=[]
path_lengths=[]
evs=[]

f = h5py.File('muon1000ev_0-1gev_edep.h5')

trajs = f['trajectories']
segs = f['segments']


primary_mask_trajs = trajs['traj_id'] == 0
muon_trajs = trajs[primary_mask_trajs] # primary muon trajectories only

for traj in muon_trajs:
    # getting event number
    ev_n = traj['event_id']
    evs.append(ev_n)
    # print('----------')
    # print(f'event: {ev_n}')

    # getting energies
    KE = traj['E_start'] - m_mu
    energies.append(KE)

    # getting naive length ---------------
    start_vector = traj['xyz_start']
    end_vector = traj['xyz_end']
    distance_vector = np.subtract(end_vector, start_vector)
    length = np.linalg.norm(distance_vector) *10 #mm
    ranges.append(length)

    # getting detailed length -------------
    # getting segments in muon track only
    ev_mask = segs['event_id'] == ev_n #ev_mask is a boolean array, true if that segment is in that trajectory
    ev_segs = segs[ev_mask] #only segments in that event
    primary_mask = ev_segs['traj_id'] == 0 
    muon_segs = ev_segs[primary_mask] #only segments in primary muon track

    path_length = 0
    for seg in muon_segs:
        path_length += seg['dx'] 
    path_lengths.append(path_length * 10)

    # getting deposited energy
    deposited_E = 0
    for seg in muon_segs:
        deposited_E += seg['dE']
    deposited_Es.append(deposited_E)

np.savez('test_muon_0-1gev_1k.npz', energies=energies, ranges=ranges, path_lengths=path_lengths, deposited_Es=deposited_Es)
